# Pix2Pix 训练实验 - Google Colab 设置

本 notebook 用于在 Google Colab 上运行 Pix2Pix (L1 + GAN) 训练实验（E2）。

## 使用说明

1. **上传代码到 GitHub/Gitee**（推荐）或直接上传 zip 到 Colab
2. **在 Google Drive 上准备数据**：将 `data/processed` 和 `data/splits` 上传到 Drive
3. **修改下面的 `DRIVE_DATA_PATH`**：改为你的 Drive 数据路径
4. **点击 "Run all"** 运行所有 cell

## 数据路径说明

- 假设你的 Drive 结构：`/content/drive/MyDrive/Cityscapes/Image-to-Image-Translation-Experiment/data`
- 需要包含：`data/processed/train/{photo,label}`, `data/processed/val/{photo,label}`, `data/splits/cityscapes_split_seed42.json`


## 1. 安装依赖


In [ ]:
# 安装必要的 Python 包
!pip install -q tqdm pillow torchvision scikit-image torchmetrics

# 如果需要 Perceptual Loss（可选）
# !pip install -q lpips

print("✅ 依赖安装完成")


## 2. 挂载 Google Drive


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

print("✅ Google Drive 挂载完成")


## 3. 克隆代码仓库（或上传代码）


In [ ]:
# 方式1：从 GitHub/Gitee 克隆（推荐）
# 请将下面的 URL 替换为你的仓库地址
# !git clone https://github.com/yourname/Image-to-Image-Translation-Experiment.git
# %cd Image-to-Image-Translation-Experiment

# 方式2：如果代码已经在 Colab 中（通过上传 zip 等方式）
# 直接使用当前目录，跳过这一步

import os
from pathlib import Path

# 设置项目根目录（根据实际情况修改）
PROJECT_ROOT = Path("/content/Image-to-Image-Translation-Experiment")

# 如果项目不在 /content 下，请修改上面的路径
# 例如：PROJECT_ROOT = Path("/content/drive/MyDrive/Image-to-Image-Translation-Experiment")

if PROJECT_ROOT.exists():
    %cd {PROJECT_ROOT}
    print(f"✅ 项目目录：{PROJECT_ROOT}")
else:
    print(f"⚠️  项目目录不存在：{PROJECT_ROOT}")
    print("请先上传代码或克隆仓库")


In [ ]:
# ⚠️ 重要：请修改下面的路径为你的 Google Drive 数据目录
# 例如：/content/drive/MyDrive/Cityscapes/Image-to-Image-Translation-Experiment/data
DRIVE_DATA_PATH = "/content/drive/MyDrive/Cityscapes/Image-to-Image-Translation-Experiment/data"

import os
from pathlib import Path

# 检查 Drive 数据路径是否存在
drive_data_path = Path(DRIVE_DATA_PATH)
if not drive_data_path.exists():
    print(f"⚠️  Drive 数据路径不存在：{DRIVE_DATA_PATH}")
    print("请检查路径是否正确，或先上传数据到 Drive")
else:
    print(f"✅ Drive 数据路径存在：{DRIVE_DATA_PATH}")
    
    # 检查必要的子目录
    required_dirs = [
        "processed/train/photo",
        "processed/train/label",
        "processed/val/photo",
        "processed/val/label",
        "splits"
    ]
    
    missing_dirs = []
    for dir_name in required_dirs:
        if not (drive_data_path / dir_name).exists():
            missing_dirs.append(dir_name)
    
    if missing_dirs:
        print(f"⚠️  缺少以下目录：{missing_dirs}")
    else:
        print("✅ 所有必要的数据目录都存在")
    
    # 检查划分文件
    split_file = drive_data_path / "splits" / "cityscapes_split_seed42.json"
    if split_file.exists():
        print(f"✅ 划分文件存在：{split_file}")
    else:
        print(f"⚠️  划分文件不存在：{split_file}")


In [ ]:
# 在项目根目录下创建指向 Drive 数据目录的软链接
import os
from pathlib import Path

project_data_dir = PROJECT_ROOT / "data"

# 如果已存在 data 目录（可能是软链接或真实目录），先删除
if project_data_dir.exists() or project_data_dir.is_symlink():
    if project_data_dir.is_symlink():
        project_data_dir.unlink()
        print(f"删除旧的软链接：{project_data_dir}")
    else:
        print(f"⚠️  {project_data_dir} 已存在且不是软链接，请手动处理")

# 创建软链接
if not project_data_dir.exists():
    os.symlink(DRIVE_DATA_PATH, project_data_dir)
    print(f"✅ 创建软链接：{project_data_dir} -> {DRIVE_DATA_PATH}")
else:
    print(f"✅ 数据目录已存在：{project_data_dir}")

# 验证软链接
if (project_data_dir / "processed" / "train" / "photo").exists():
    print("✅ 数据软链接验证成功")
    train_photos = list((project_data_dir / 'processed' / 'train' / 'photo').glob('*.jpg'))
    val_photos = list((project_data_dir / 'processed' / 'val' / 'photo').glob('*.jpg'))
    print(f"   训练集 photo 数量：{len(train_photos)} 张")
    print(f"   验证集 photo 数量：{len(val_photos)} 张")
else:
    print("⚠️  数据软链接验证失败，请检查路径")


In [ ]:
import sys
from pathlib import Path

# 将项目根目录添加到 Python 路径
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"✅ Python 路径已设置")
print(f"   项目根目录：{PROJECT_ROOT}")
print(f"   sys.path[0]：{sys.path[0]}")

# 验证导入
try:
    from src.models.generator import UNetGenerator
    from src.models.discriminator import PatchGANDiscriminator
    from src.losses.pix2pix_losses import pix2pix_generator_loss
    print("✅ 模块导入成功")
except ImportError as e:
    print(f"⚠️  模块导入失败：{e}")
    print("请检查项目结构是否正确")


## 7. 检查 GPU 可用性


In [ ]:
import torch

print(f"PyTorch 版本：{torch.__version__}")
print(f"CUDA 可用：{torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA 版本：{torch.version.cuda}")
    print(f"GPU 设备：{torch.cuda.get_device_name(0)}")
    print(f"GPU 数量：{torch.cuda.device_count()}")
    print("✅ GPU 可用，训练将使用 GPU")
else:
    print("⚠️  GPU 不可用，训练将使用 CPU（会很慢）")
    print("建议：在 Colab 中点击 Runtime -> Change runtime type -> Hardware accelerator -> GPU")


## 8. 运行 E2 实验：Pix2Pix (L1 + GAN)

### 8.1 快速测试（5 epochs，用于验证环境）


In [ ]:
# 快速测试：运行 5 个 epoch 验证环境是否正确
%cd {PROJECT_ROOT}

!python src/training/train_pix2pix.py \
  --data-root data \
  --split-index data/splits/cityscapes_split_seed42.json \
  --epochs 5 \
  --batch-size 1 \
  --exp-name pix2pix_l1_gan_strong_debug \
  --aug-mode strong \
  --lambda-l1 100 \
  --num-val-samples 5

print("\n✅ 快速测试完成")


### 8.2 正式训练（200 epochs，完整 E2 实验）

**注意**：200 epochs 训练时间较长（根据 GPU 性能，可能需要数小时）。

建议：
- 使用 Colab Pro 或更高版本以获得更长的运行时间
- 定期保存 checkpoint（脚本已自动保存）
- 如果 Colab 会话中断，可以从 checkpoint 恢复训练（需要修改脚本支持 `--resume`）


In [ ]:
# 正式 E2 实验：200 epochs
%cd {PROJECT_ROOT}

!python src/training/train_pix2pix.py \
  --data-root data \
  --split-index data/splits/cityscapes_split_seed42.json \
  --epochs 200 \
  --batch-size 1 \
  --exp-name pix2pix_l1_gan_strong \
  --aug-mode strong \
  --lambda-l1 100 \
  --lr 2e-4 \
  --start-decay-epoch 100 \
  --num-val-samples 10 \
  --save-interval 10

print("\n✅ E2 实验训练完成")


## 9. 保存结果到 Google Drive（可选）


In [ ]:
# 将训练结果（checkpoints、images、logs）复制到 Google Drive
# 这样即使 Colab 会话结束，结果也不会丢失

import shutil
from pathlib import Path

outputs_dir = PROJECT_ROOT / "outputs"
drive_outputs_dir = Path("/content/drive/MyDrive/Cityscapes/Image-to-Image-Translation-Experiment/outputs")

if outputs_dir.exists():
    # 创建 Drive 输出目录
    drive_outputs_dir.mkdir(parents=True, exist_ok=True)
    
    # 复制整个 outputs 目录到 Drive
    print(f"正在复制结果到 Drive：{drive_outputs_dir}")
    
    # 逐个复制子目录
    for exp_dir in outputs_dir.iterdir():
        if exp_dir.is_dir():
            dest_dir = drive_outputs_dir / exp_dir.name
            if dest_dir.exists():
                print(f"  跳过已存在的目录：{dest_dir.name}")
            else:
                shutil.copytree(exp_dir, dest_dir)
                print(f"  ✅ 已复制：{exp_dir.name}")
    
    print(f"\n✅ 结果已保存到 Drive：{drive_outputs_dir}")
else:
    print(f"⚠️  输出目录不存在：{outputs_dir}")
    print("请先运行训练脚本")


In [ ]:
# 查看训练历史（loss 曲线等）
import json
from pathlib import Path
import matplotlib.pyplot as plt

history_file = PROJECT_ROOT / "outputs" / "pix2pix_l1_gan_strong" / "logs" / "history_pix2pix.json"

if history_file.exists():
    with open(history_file, 'r', encoding='utf-8') as f:
        history = json.load(f)
    
    print("训练历史摘要：")
    print(f"  总 epoch 数：{len(history.get('train_g_total', []))}")
    if history.get('val_psnr'):
        print(f"  最佳 Val PSNR：{max(history.get('val_psnr', [0])):.4f}")
    if history.get('val_ssim'):
        print(f"  最佳 Val SSIM：{max(history.get('val_ssim', [0])):.4f}")
    if history.get('val_l1'):
        print(f"  最低 Val L1：{min(history.get('val_l1', [float('inf')])):.4f}")
    
    # 绘制损失曲线（简单示例）
    epochs = list(range(1, len(history.get('train_g_total', [])) + 1))
    
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    if history.get('train_g_total'):
        plt.plot(epochs, history.get('train_g_total', []), label='G Total')
    if history.get('train_g_l1'):
        plt.plot(epochs, history.get('train_g_l1', []), label='G L1')
    if history.get('train_g_gan'):
        plt.plot(epochs, history.get('train_g_gan', []), label='G GAN')
    if history.get('train_d_loss'):
        plt.plot(epochs, history.get('train_d_loss', []), label='D Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Training Losses')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(1, 2, 2)
    if history.get('val_psnr'):
        plt.plot(epochs, history.get('val_psnr', []), label='PSNR')
    if history.get('val_ssim'):
        plt.plot(epochs, history.get('val_ssim', []), label='SSIM')
    plt.xlabel('Epoch')
    plt.ylabel('Metric')
    plt.title('Validation Metrics')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()
else:
    print(f"⚠️  训练历史文件不存在：{history_file}")
